In [1]:
from _Setup import *

In [2]:
train = pd.read_csv(missForestTrainFile)
test = pd.read_csv(missForestTestFile)
submission_data = pd.read_csv(rawSubmissionFile)

In [11]:
#Separating columns 
#X = train.drop(columns=['Unit2','Unnamed: 0', 'HospAdmTime','SepsisLabel', 'time']) 
X = train.drop(columns=['Unit2','Unnamed: 0', 'HospAdmTime', 'time'])  
y = train['SepsisLabel']

In [12]:
X.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,FiO2,pH,...,Hct,Hgb,WBC,Platelets,Age,patient,Gender,Unit1,ICULOS,SepsisLabel
0,-0.432220,0.749512,-2.124421,-0.621283,0.146051,0.420292,-1.294263,0.045536,0.073114,0.340467,...,0.422033,0.740608,0.373687,-0.431184,-0.299413,16426,1.0,0.0,1.0,0.0
1,-0.432220,0.749512,-2.124421,-0.621283,0.146051,0.420292,-1.294263,0.045536,0.073114,0.340467,...,0.422033,0.740608,0.373687,-0.431184,-0.299413,16426,1.0,0.0,2.0,0.0
2,-0.320708,0.592277,-0.914129,-0.428503,0.299849,0.716228,-1.294263,0.045536,-0.000054,0.710267,...,0.422033,0.740608,0.373687,-0.431184,-0.299413,16426,1.0,0.0,3.0,0.0
3,-0.376464,0.906747,-0.531932,-0.471343,0.053772,0.346308,-1.294263,-0.768098,-0.000054,-0.029333,...,0.422033,0.740608,0.373687,-0.431184,-0.299413,16426,1.0,0.0,4.0,0.0
4,-0.710999,0.906747,-0.531932,-0.342823,-0.130785,0.050372,-1.479445,-0.768098,-0.014687,-0.029333,...,0.422033,0.740608,0.373687,-0.431184,-0.299413,16426,1.0,0.0,5.0,0.0


In [5]:
#k = 10  # Adjust this number based on your needs
#selector = SelectKBest(score_func=f_classif, k=k)
#X_selected = selector.fit_transform(X, y)

# Get the names of the selected features
#selected_features = X.columns[selector.get_support()]
#print("Selected Features:", selected_features)

In [13]:
# take a sample from the data 
sampled_patient_ids = X['patient'].drop_duplicates().sample(frac=0.1, random_state=42) 

# Filter the original DataFrame to include only the sampled patient IDs
sampled_data = X[X['patient'].isin(sampled_patient_ids)]
y = sampled_data['SepsisLabel']

In [14]:
selected_columns = ['ICULOS']
X_selected = sampled_data[selected_columns]

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='linear') 
model.fit(X_selected, y)

In [ ]:
y_pred = model.predict(test)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
}

grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_selected, y)

print(f"Best parameters: {grid_search.best_params_}")

In [ ]:
submission_data['SepsisLabel'] = y_pred

# Optionally, save the modified DataFrame to a CSV file
# TODO : create a variable for this path in _Setup.py
# submission_data.to_csv('submission_data_with_predictions.csv', index=False)

# Display the updated DataFrame (optional)
print(submission_data.head())

In [ ]:
num_rows = submission_data.shape[0]
print("Number of rows in submission_data:", num_rows)

In [ ]:
print("\nClassification Report (Training):\n", classification_report(y, y_pred))

In [ ]:
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")

## Logistic regression 

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [53]:
columns_t = ['ICULOS', 'HR', 'O2Sat', 'Temp', 'Resp', 'BaseExcess', 'BUN', 'FiO2', 'Magnesium', 'Age', 'WBC', 'Platelets' , 'Calcium', 'Creatinine', 
            'MAP', 'SBP' , 'Hct', 'PaCO2']
columns_test = ['ICULOS', 'HR', 'O2Sat', 'Temp', 'Resp', 'BaseExcess', 'BUN', 'FiO2', 'Magnesium', 'Age', 'WBC', 'Platelets' , 'Calcium', 'Creatinine', 
            'MAP', 'SBP' , 'Hct', 'PaCO2']

x_train = train[columns_t]
x_test = test[columns_test]

In [63]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

x_train_sm = sm.add_constant(X)
model = sm.Logit(y, x_train_sm)
result = model.fit()
y_pred_prob = result.predict(x_train_sm)  
y_pred = (y_pred_prob >= 0.4).astype(int)  

# Calculate the confusion matrix
cm_train = confusion_matrix(y, y_pred)


# Print the summary
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.469206
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:            SepsisLabel   No. Observations:              1389147
Model:                          Logit   Df Residuals:                  1389120
Method:                           MLE   Df Model:                           26
Date:                Thu, 31 Oct 2024   Pseudo R-squ.:                  0.1314
Time:                        11:04:02   Log-Likelihood:            -6.5180e+05
converged:                       True   LL-Null:                   -7.5042e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.0513      0.006   -344.410      0.000      -2.063      -2.040
HR             0.2457      0.

In [64]:
cm_train = confusion_matrix(y, y_pred)

# Print the confusion matrix
print("Confusion Matrix (Training Data):")
print(cm_train)

Confusion Matrix (Training Data):
[[1035086   33489]
 [ 250863   69709]]


In [65]:
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score

f1 = f1_score(y, y_pred)
print(f"F1 Score (Training Data): {f1:.2f}")

F1 Score (Training Data): 0.33


In [57]:
submission_data['SepsisLabel'] = y_pred

# Optionally, save the modified DataFrame to a CSV file
# TODO : create a variable for this path in _Setup.py
# submission_data.to_csv('submission_data_with_predictions.csv', index=False)

# Display the updated DataFrame (optional)
print(submission_data.head())

       ID  SepsisLabel
0  6560_1            0
1  6560_2            0
2  6560_3            0
3  6560_4            0
4  6560_5            0


In [60]:
num_rows = submission_data.shape[0]
print("Number of rows in submission_data:", num_rows)

Number of rows in submission_data: 464354


In [61]:
submission_data.to_csv('submission_data_Logit.csv', index=False)

## Random Forest 

In [67]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)

# Make predictions on the test set
y_pred = model.predict(test)

# Evaluate the model
cm = confusion_matrix(y, y_pred)
print("Confusion Matrix (Test Data):")
print(cm)

# Print metrics
accuracy = accuracy_score(y, y_pred)
f1 = f1_score(y, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")